In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

# Zmiana Typow

Aby sprawdzic typ kolumny w Pandas, można użyć następujących metod:
* dataFrame.dtypes
* dataFrame.info()  
* dataFrame.<columna>.dtype

Zmiana typu kolumny w Pandas można wykonać na kilka sposobów:

* pd.to_datetime https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html
* pd.to_numeric https://pandas.pydata.org/docs/reference/api/pandas.to_numeric.html
* pd.to_timedelta https://pandas.pydata.org/docs/reference/api/pandas.to_timedelta.html
* <dataFrame>[<columna>].astype(<nazwa typu>)

Dokumentacja:
 https://pandas.pydata.org/docs/user_guide/basics.html#basics-dtypes
 https://pandas.pydata.org/docs/reference/arrays.html#api-arrays-integer-na


In [21]:
s = pd.Series([1,2,3,4,np.nan])
print('Typ wybrany przez Pandasa: ', s.dtypes)

print('------------------------------------------')

print("nie mozna zmienic na int - nie ma w nim nan")
try:
    s = s.astype('int64') ## int z malej litery nie dziala - nie ma nan
    print(s.dtypes)
except Exception as e:
    print(e)

print('------------------------------------------')
print("Zmiana na Int - dziala")

s = s.astype('Int64') # trzeba uzyc Int z duzej litery - 
print(s.dtypes)
print(s)

Typ wybrany przez Pandasa:  float64
nie mozna zmienic na int - nie ma w nim nan
Cannot convert non-finite values (NA or inf) to integer
Zmiana na Int - dziala
Int64
0       1
1       2
2       3
3       4
4    <NA>
dtype: Int64


In [22]:
## mozna tez podac typ przy tworzeniu zmiennej
s = pd.Series([1,2,3,4,np.nan], dtype='Int64')
print('Typ wybrany przez Pandasa: ', s.dtypes)

Typ wybrany przez Pandasa:  Int64


# pd.Series.str
https://pandas.pydata.org/docs/reference/api/pandas.Series.str.html#pandas.Series.str

umozliwia wykorzystywanie metod stringa w sposob zwektoryzowany (czyli szybszy obliczeniowo)

In [10]:
s = pd.Series(["A_Str_Series", "other_str_siries", "two_parts"])

In [8]:
s.str.len()

0    12
1    16
dtype: int64

In [7]:
s.str.split('_')

0        [A, Str, Series]
1    [other, str, siries]
dtype: object

In [11]:
s.str.split('_', expand=True)

,0,1,2
0,A,Str,Series
1,other,str,siries
2,two,parts,None


In [14]:
## alternatywne rozwiazanie: 
for index, value in s.items():
    print(index, value.split("_"))

0 ['A', 'Str', 'Series']
1 ['other', 'str', 'siries']
2 ['two', 'parts']


In [6]:
df = s.str.split('_', expand=True)
df

## jezeli chcemy je polaczyc spowrotem:

df['combined'] = df[0] + '_' + df[1] + '_' + df[2]
display(df)


NameError: name 's' is not defined

#### Apply
https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html#pandas.DataFrame.apply

Apply a function along an axis of the DataFrame.

axis{0 or ‘index’, 1 or ‘columns’}, default 0
Axis along which the function is applied:

* 0 or ‘index’: apply function to each column.

* 1 or ‘columns’: apply function to each row.

In [ ]:
df = s.str.split('_', expand=True)
df['combined'] = df.apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
# lub 
df['combined'] = df[[0,1,2]].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)

df['combined'] = df[[0,1,2]].apply(lambda row: '_'.join(row[row.notna()].values.astype(str)), axis=1)
df

,0,1,2,combined
0,A,Str,Series,A_Str_Series
1,other,str,siries,other_str_siries
2,two,parts,None,two_parts


# Groupby

In [30]:


df_pingwinki = sns.load_dataset('penguins')


### Metoda 1:
* `df.groupby(<column(s)>).<funkcja>`
* `df.groupby(<column(s)>)[<column(s)>].<funkcja>`


In [4]:
## liczba obserwacji na kazdej wyspie
df_pingwinki.groupby('island').size()

island
Biscoe       168
Dream        124
Torgersen     52
dtype: int64

In [5]:
## ale juz count, mean, sum... podsumuje kazda kolumne
df_pingwinki.groupby('island').count() 

,species,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
island,,,,,,
Biscoe,168,167,167,167,167,163
Dream,124,124,124,124,124,123
Torgersen,52,51,51,51,51,47


In [7]:
## mean zwroci blad bo niektore kolumny sa tekstowe
df_pingwinki.groupby('island').mean() 

TypeError: agg function failed [how->mean,dtype->object]

**Cwiczenie:**

jak zadziala sum?

In [9]:
df_pingwinki.groupby('island')[['bill_length_mm', 'bill_depth_mm',
       'flipper_length_mm', 'body_mass_g']].mean() 

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
island,,,,
Biscoe,45.257485,15.874850,209.706587,4716.017964
Dream,44.167742,18.344355,193.072581,3712.903226
Torgersen,38.950980,18.429412,191.196078,3706.372549


### Metoda 2:
* `df.groupby(<column(s)>).agg({'kolumna': ['funkcja1', 'funkcja2', ..], ... })`

In [11]:
df_gr = df_pingwinki.groupby('island').agg({'bill_length_mm': ['mean', 'sum'],
                                   'species': ['nunique'],
                                   'sex': ['count']})

In [13]:
df_gr

bill_length_mm         species   sex
                    mean     sum nunique count
island                                        
Biscoe         45.257485  7558.0       2   163
Dream          44.167742  5476.8       2   123
Torgersen      38.950980  1986.5       1    47

In [14]:
df_gr.columns

MultiIndex([('bill_length_mm',    'mean'),
            ('bill_length_mm',     'sum'),
            (       'species', 'nunique'),
            (           'sex',   'count')],
           )

In [16]:
print(df_gr.loc['Biscoe'])
display(df_gr.loc[['Biscoe']])

bill_length_mm  mean         45.257485
                sum        7558.000000
species         nunique       2.000000
sex             count       163.000000
Name: Biscoe, dtype: float64


bill_length_mm         species   sex
                 mean     sum nunique count
island                                     
Biscoe      45.257485  7558.0       2   163

In [45]:
df_gr['bill_length_mm','mean']

island
Biscoe       45.257485
Dream        44.167742
Torgersen    38.950980
Name: (bill_length_mm, mean), dtype: float64

In [17]:
df_gr.loc['Biscoe', ('bill_length_mm','mean')]

np.float64(45.25748502994012)

In [18]:
df_gr.columns = ['_'.join(col) for col in df_gr.columns]

### Metoda 3:

```<dataFrame>.groupby([<column(s)>]).agg(nowa_nazwa_kolumny = ('<kolumna z df>', 'funkcja agg), ...)```



In [19]:
## mozemy tez zdefiniowac wlasna fuckcje: musi ona brac jako argument pd.Series i zwracac wartosc zagregowana np:

def count_female(x):
    return (x=='Female').sum()

In [20]:
df_pingwinki.groupby('island').agg(bill_length_mean = ('bill_length_mm', 'mean'),
                                   unique_species = ('species', 'nunique'),
                                   count = ('sex', 'count'),
                                   count_female = ('sex', count_female))

,bill_length_mean,unique_species,count,count_female
island,,,,
Biscoe,45.257485,2,163,80
Dream,44.167742,2,123,61
Torgersen,38.950980,1,47,24


In [21]:
def count_sex(x, sex):
    return (x==sex).sum()

df_pingwinki.groupby('island').agg(bill_length_mean = ('bill_length_mm', 'mean'),
                                   unique_species = ('species', 'nunique'),
                                   count = ('sex', 'count'),
                                   count_female = ('sex', lambda x: count_sex(x, 'Female')),
                                   count_male = ('sex', lambda x: count_sex(x, 'Male'))
                                  )

,bill_length_mean,unique_species,count,count_female,count_male
island,,,,,
Biscoe,45.257485,2,163,80,83
Dream,44.167742,2,123,61,62
Torgersen,38.950980,1,47,24,23


### Co jeszcze warto wiedziec

In [22]:
## aby pokazac nan:

df_pingwinki.groupby('sex', dropna=False).size()

sex
Female    165
Male      168
NaN        11
dtype: int64

In [41]:
## zmienmy wyspe na zmienna kategoryczna:
df_pingwinki['island_cat'] = pd.Categorical(df_pingwinki['island'])

In [25]:
display(df_pingwinki.groupby(['species', 'island_cat'], observed=True).size())
display(df_pingwinki.groupby(['species', 'island_cat'], observed=False).size())


## warning jest zwiazany z nadchodzaca zmiana w pandas 3

species    island_cat
Adelie     Biscoe         44
           Dream          56
           Torgersen      52
Chinstrap  Dream          68
Gentoo     Biscoe        124
dtype: int64

species    island_cat
Adelie     Biscoe         44
           Dream          56
           Torgersen      52
Chinstrap  Biscoe          0
           Dream          68
           Torgersen       0
Gentoo     Biscoe        124
           Dream           0
           Torgersen       0
dtype: int64

In [26]:
df_pingwinki.query('species=="Chinstrap"').groupby('island', observed=True).size() 

island
Dream    68
dtype: int64

In [27]:
df_pingwinki.query('species=="Chinstrap"').groupby('island_cat', observed=False).size() 

island_cat
Biscoe        0
Dream        68
Torgersen     0
dtype: int64


**Cwiczenie** 

Policz ile pingwinkow ma bill_length_mm wieksze od 40, oraz jaka jest minimalna i maksymalna masa ciala w podziale na wyspe i gatunek.
* ile jest takich pingwinkow z wyspy  'Biscoe' z gatunku 'Adelie' (oczywiscie chodzi o wypisanie wyniku w postaci jednej liczby a nie przeczytanie z tabeli.)


In [45]:
import seaborn as sns
import pandas as pd
import numpy as np

df_pingwinki = sns.load_dataset('penguins')
df_pingwinki['island_cat'] = pd.Categorical(df_pingwinki['island'])

print('wiekszy dziob niz 40', df_pingwinki.query('bill_length_mm > 40').shape[0])
print('najmniejsze cialko', df_pingwinki['body_mass_g'].min())
print('najwieksze cialko', df_pingwinki['body_mass_g'].max())
print('wiekszy dziob niz 40 na Adelie', 
      df_pingwinki
        .query('bill_length_mm > 40')
        .query('species == "Adelie"')
        .query('island_cat == "Biscoe"')
        .shape[0])

tmp_df = df_pingwinki.groupby(['species', 'island_cat'], observed=True).agg(
    body_mass_min=('body_mass_g', 'min'),
    body_mass_max=('body_mass_g', 'max'),
    bill_bigger_than_40=('bill_length_mm', lambda x: (x > 40).sum())
)
print('wiekszy dziob niz 40 na Adelie',  tmp_df.loc[('Adelie', 'Biscoe'), 'bill_bigger_than_40'])
tmp_df

wiekszy dziob niz 40 242
najmniejsze cialko 2700.0
najwieksze cialko 6300.0
wiekszy dziob niz 40 na Adelie 16
wiekszy dziob niz 40 na Adelie 16


body_mass_min  body_mass_max  bill_bigger_than_40
species   island_cat                                                   
Adelie    Biscoe             2850.0         4775.0                   16
          Dream              2900.0         4650.0                   17
          Torgersen          2900.0         4700.0                   18
Chinstrap Dream              2700.0         4800.0                   68
Gentoo    Biscoe             3950.0         6300.0                  123

In [19]:
df_pingwinki.groupby(['species','sex'])['island_cat'].nunique()


species    sex   
Adelie     Female    3
           Male      3
Chinstrap  Female    1
           Male      1
Gentoo     Female    1
           Male      1
Name: island_cat, dtype: int64

In [20]:
df_pingwinki.groupby(['species','sex'])['body_mass_g'].mean()


species    sex   
Adelie     Female    3368.835616
           Male      4043.493151
Chinstrap  Female    3527.205882
           Male      3938.970588
Gentoo     Female    4679.741379
           Male      5484.836066
Name: body_mass_g, dtype: float64

In [21]:
df_pingwinki.groupby(['species','sex'])['body_mass_g'].sum()


species    sex   
Adelie     Female    245925.0
           Male      295175.0
Chinstrap  Female    119925.0
           Male      133925.0
Gentoo     Female    271425.0
           Male      334575.0
Name: body_mass_g, dtype: float64

In [22]:
df_pingwinki.groupby(['species', 'sex'])[['island_cat']].size()


species    sex   
Adelie     Female    73
           Male      73
Chinstrap  Female    34
           Male      34
Gentoo     Female    58
           Male      61
dtype: int64

In [23]:
df_pingwinki.groupby(['species', 'sex']).agg({
    'island': ['nunique', 'count'],
    'body_mass_g': ['mean', 'sum']
})

island        body_mass_g          
                 nunique count         mean       sum
species   sex                                        
Adelie    Female       3    73  3368.835616  245925.0
          Male         3    73  4043.493151  295175.0
Chinstrap Female       1    34  3527.205882  119925.0
          Male         1    34  3938.970588  133925.0
Gentoo    Female       1    58  4679.741379  271425.0
          Male         1    61  5484.836066  334575.0

In [87]:
df_pingwinki

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,island_cat
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male,Torgersen
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female,Torgersen
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female,Torgersen
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,Torgersen
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female,Torgersen
...,...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN,Biscoe
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female,Biscoe
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male,Biscoe
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female,Biscoe


## Merge
* https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html#pandas.DataFrame.merge

* istnieje tez `join` ale umozliwia laczenie tylko po wspolnej kolumnie https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html#pandas.DataFrame.join

In [48]:
df_mean_mass = df_pingwinki.groupby(['species'])[['body_mass_g']].agg('mean')

In [49]:
df_pingwinki.merge(df_mean_mass, left_on=['species'], right_index=True)


,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g_x,sex,island_cat,body_mass_g_y
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male,Torgersen,3700.662252
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female,Torgersen,3700.662252
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female,Torgersen,3700.662252
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,Torgersen,3700.662252
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female,Torgersen,3700.662252
...,...,...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN,Biscoe,5076.016260
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female,Biscoe,5076.016260
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male,Biscoe,5076.016260
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female,Biscoe,5076.016260


**Cwiczenie**

Policz srednia mase pingwikow w podziale na gatunek i plec i polacz dane z wyjsciowa tabele df_pingwinki.
* Czy po mergu liczba wierszy jest taka sama jak w wyjsciowej tabeli?



In [ ]:
# mean body mass by species and sex
df_mean_mass_species_sex = df_pingwinki.groupby(['species', 'sex'])[['body_mass_g']].mean()
df_pingwinki = df_pingwinki.merge(
    df_mean_mass_species_sex.rename(columns={'body_mass_g': 'mean_body_mass_species_sex'}).reset_index(),
    on=['species', 'sex'],
    how='left'
)
df_pingwinki

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,island_cat,mean_body_mass_species_sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male,Torgersen,4043.493151
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female,Torgersen,3368.835616
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female,Torgersen,3368.835616
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN,Torgersen,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female,Torgersen,3368.835616
...,...,...,...,...,...,...,...,...,...
339,Gentoo,Biscoe,NaN,NaN,NaN,NaN,NaN,Biscoe,NaN
340,Gentoo,Biscoe,46.8,14.3,215.0,4850.0,Female,Biscoe,4679.741379
341,Gentoo,Biscoe,50.4,15.7,222.0,5750.0,Male,Biscoe,5484.836066
342,Gentoo,Biscoe,45.2,14.8,212.0,5200.0,Female,Biscoe,4679.741379
